In [65]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors

import glob
from PIL import Image
from scipy import interpolate
from scipy import stats
import matplotlib.ticker as ticker
import warnings

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format

In [111]:
# Get all the csv files in the current directory
csv_files = glob.glob("../../experiments/logs/simdata/table1/volterra/exp_*.csv")

# Create a list of dataframes
dfs = []

# Iterate over the csv files
for csv_file in csv_files:

    # Read the csv file into a dataframe
    df = pd.read_csv(csv_file)

    # Append the dataframe to the list of dataframes
    dfs.append(df)

# Concatenate the dataframes into a single dataframe
df = pd.concat(dfs)
df

,A_000,A_001,A_002,A_010,A_011,A_012,A_020,A_021,A_022,A_100,...,generation_model,plasticity_coeff_init,plasticity_model,moving_avg_window,data_dir,log_dir,trainable_coeffs,exp_name,reward_term,layer_sizes
0,0.008,0.006,0.000,0.009,0.001,0.000,0.008,0.008,0.000,0.008,...,volterra,random,volterra,10,../data/,logs/,18,table1,expected-reward,"[2, 10, 1]"
1,0.024,0.035,0.000,0.070,0.000,0.000,0.024,0.024,0.000,0.022,...,volterra,random,volterra,10,../data/,logs/,18,table1,expected-reward,"[2, 10, 1]"
2,0.030,0.047,0.000,0.101,0.000,0.000,0.030,0.010,0.000,0.027,...,volterra,random,volterra,10,../data/,logs/,18,table1,expected-reward,"[2, 10, 1]"
3,0.036,0.056,0.000,0.119,0.000,0.000,0.035,0.000,0.000,0.030,...,volterra,random,volterra,10,../data/,logs/,18,table1,expected-reward,"[2, 10, 1]"
4,0.042,0.060,0.000,0.126,0.000,0.000,0.037,0.000,0.000,0.033,...,volterra,random,volterra,10,../data/,logs/,18,table1,expected-reward,"[2, 10, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,0.055,0.028,0.000,0.043,0.000,0.000,0.043,0.000,0.000,0.040,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
744,0.062,0.017,0.000,0.047,0.000,0.000,0.047,0.000,0.000,0.043,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
745,0.069,0.003,0.000,0.051,-0.000,0.000,0.051,0.000,0.000,0.044,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
746,0.076,0.000,0.000,0.054,0.000,0.000,0.054,0.000,0.000,0.045,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"


In [112]:
coeffs, other_columns = [], []

for i in range(3):
        for j in range(3):
            for k in range(3):
                coeffs.append(f"A_{i}{j}{k}")

for column in df.columns:
    if column not in set(coeffs):
        other_columns.append(column)
print(other_columns)
num_epochs = 250

if df["plasticity_model"].unique()[0] == "volterra":
    for coeff in coeffs:
        df[coeff] = pd.to_numeric(df[coeff])

df["moving_avg_window"] = df["moving_avg_window"].astype(int)
df["flyid"].unique()

['epoch', 'loss', 'train_time', 'percent_deviance', 'r2_weights', 'r2_activity', 'num_train', 'num_eval', 'num_epochs', 'trials_per_block', 'log_interval', 'num_blocks', 'log_expdata', 'use_experimental_data', 'flyid', 'fit_data', 'input_firing_mean', 'input_variance', 'l1_regularization', 'generation_coeff_init', 'generation_model', 'plasticity_coeff_init', 'plasticity_model', 'moving_avg_window', 'data_dir', 'log_dir', 'trainable_coeffs', 'exp_name', 'reward_term', 'layer_sizes']


array([ 1, 10,  2,  3,  4,  5,  6,  7,  8,  9])

In [113]:
df_subset = df.loc[(df['epoch'] == num_epochs) & (df['reward_term'] == "reward")]
df_subset.head()

,A_000,A_001,A_002,A_010,A_011,A_012,A_020,A_021,A_022,A_100,...,generation_model,plasticity_coeff_init,plasticity_model,moving_avg_window,data_dir,log_dir,trainable_coeffs,exp_name,reward_term,layer_sizes
219,0.091,0.000,0.000,0.056,0.000,0.000,0.056,0.000,0.000,0.041,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
230,0.085,0.000,0.000,0.051,0.000,0.000,0.051,0.000,0.000,0.032,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
241,0.040,-0.000,0.000,0.029,-0.000,0.000,0.029,-0.000,0.000,0.000,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
252,0.044,-0.000,0.000,0.033,-0.000,0.000,0.033,0.000,0.000,0.000,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"
263,0.044,-0.000,0.000,0.033,-0.000,0.000,0.033,0.000,0.000,0.000,...,volterra,random,volterra,10,../data/,logs/,18,table1,reward,"[2, 10, 1]"


In [114]:
df_plot = df_subset[["r2_weights", "r2_activity", "percent_deviance", "generation_coeff_init"]]
df_plot = df_plot.groupby('generation_coeff_init')[["r2_weights", "r2_activity", "percent_deviance"]].mean()
# df_plot = df_plot.loc[(df_plot['r2_weights'] > 0)]

In [115]:
df_plot

,r2_weights,r2_activity,percent_deviance
generation_coeff_init,,,
X0R0W0,-14.639,-0.724,90.795
X0R0W0-0.05X0R0W1,-997.517,-0.682,90.795
X0R0W0-0.05X0R0W2,-99211.643,-0.668,90.795
X0R0W0-0.05X1R0W1,-114.132,-0.709,90.795
X0R1W0,-9.386,0.215,90.242
X0R1W0-0.05X0R0W1,-605.148,0.248,90.155
X0R1W0-0.05X0R0W2,-11929.542,0.231,90.371
X0R1W0-0.05X1R0W1,-73.039,0.226,90.236
X0R2W0,-9.386,0.215,90.242
